## Start

In [2]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[total_active_users_per_day_full_table]]

### Table

In [6]:
query_text = """--sql
CREATE TABLE db1.total_active_users_per_day_rep_mobile_total
(
    `report_date` Date,
    `partner_uuid` String,
    `city` String,
    `total_active_users_per_day` Int32,
    `total_active_user_monetization_per_day` Int32,
    `total_active_user_subscribed_monetization_per_day` Int32
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [7]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.total_active_users_per_day_rep_mobile_total_mv
    REFRESH EVERY 1 DAY OFFSET 6 HOUR 5 MINUTE TO db1.total_active_users_per_day_rep_mobile_total AS
    SELECT
        report_date,
        partner_uuid,
        city,
        round(avg(citizen_id_count) OVER (PARTITION BY partner_uuid, city, toStartOfMonth(report_date)
            ORDER BY report_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )) AS total_active_users_per_day,
        round(avg(total_active_user_monetization) OVER (PARTITION BY partner_uuid, city, toStartOfMonth(report_date)
            ORDER BY report_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )) AS total_active_user_monetization_per_day,
        round(avg(total_active_user_subscribed_monetization) OVER (PARTITION BY partner_uuid, city, toStartOfMonth(report_date)
            ORDER BY report_date
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )) AS total_active_user_subscribed_monetization_per_day
    FROM
        (SELECT
            report_date,
            partner_uuid,
            city,
            count(citizen_id) as citizen_id_count,
            count(if(monetization = 1,citizen_id,null)) as total_active_user_monetization,
            count(if(monetization = 1 and subscriptions_state = 'activated',citizen_id,null)) as total_active_user_subscribed_monetization
        FROM db1.total_active_users_per_day_full_table
        GROUP BY report_date,
                partner_uuid,
                city
        )
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [10]:
query_text = """--sql
    SELECT
        *
    FROM db1.total_active_users_per_day_rep_mobile_total
    WHERE city !=''
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,city,total_active_users_per_day,total_active_user_monetization_per_day,total_active_user_subscribed_monetization_per_day
date,str,str,i64,i64,i64
2025-08-03,"""""","""Одинцово""",1,1,0
2025-08-03,"""""","""Иннополис""",1,1,0
2025-08-03,"""""","""Агидель""",1,1,0
2025-08-03,"""""","""Новокузнецк""",22,22,0
2025-08-03,"""""","""Махачкала""",1,1,1
…,…,…,…,…,…
2025-08-03,"""17a5ae08-73e6-43fd-a82b-c8a764…","""Ульяновск""",27,27,8
2025-08-03,"""16fdef47-f0b4-4ba8-a1a2-8bcb2b…","""Ангарск""",302,302,47
2025-08-03,"""1854336c-9a24-4674-8b9d-446e99…","""Бишкек""",2022,1305,1


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.total_activated_account_rep_mobile_full DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.total_activated_account_rep_mobile_full_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.total_activated_account_rep_mobile_full
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.total_activated_account_rep_mobile_full_mv
"""

ch.query_run(query_text)